In [58]:
import numpy as np 
import pandas as pd 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples,silhouette_score
import matplotlib.pyplot as plt 
%matplotlib inline

dropouts = pd.read_csv(r"dropouts.csv", index_col=0)
enrollments = pd.read_csv(r"enrollments.csv", index_col=0)
factors = pd.read_csv(r"factors.csv", index_col=0)

dropouts_clusters = pd.read_csv(r"dropouts_clusters.csv", index_col=0)
enrollments_clusters = pd.read_csv(r"enrollments_clusters.csv", index_col=0)
factors_clusters = pd.read_csv(r"factors_clusters.csv", index_col=0)

In [59]:
factors.State_UT

0     Andaman & Nicobar Islands
1                Andhra Pradesh
2             Arunachal Pradesh
3                         Assam
4                         Bihar
5                    Chandigarh
6                  Chhattisgarh
7          Dadra & Nagar Haveli
8                   Daman & Diu
9                         Delhi
10                          Goa
11                      Gujarat
12                      Haryana
13             Himachal Pradesh
14            Jammu And Kashmir
15                    Jharkhand
16                    Karnataka
17                       Kerala
18                  Lakshadweep
19               Madhya Pradesh
20                  Maharashtra
21                      Manipur
22                    Meghalaya
23                      Mizoram
24                     Nagaland
25                       Odisha
26                   Puducherry
27                       Punjab
28                    Rajasthan
29                       Sikkim
30                   Tamil Nadu
31      

In [60]:
dropouts.State.str.strip()
dropouts.State.replace({'A & N Islands':'Andaman & Nicobar Islands','Arunachal  Pradesh':'Arunachal Pradesh','Jammu & Kashmir':'Jammu And Kashmir','Madhya  Pradesh':'Madhya Pradesh','Pondicherry':'Puducherry','Tamil  Nadu':'Tamil Nadu','Uttaranchal':'Uttarakhand'},inplace=True)
enrollments.State.replace({'Uttaranchal':'Uttarakhand','Pondicherry':'Puducherry'},inplace=True)

In [61]:
enrollments.set_index('State',inplace=True)
dropouts.set_index('State',inplace=True)
factors.set_index('State_UT',inplace=True)

In [62]:
dataset = pd.concat([dropouts,enrollments,factors],axis=1)
dataset

,PRIMARY_TOTAL,UPPER_PRIMARY_TOTAL,SECONDARY_TOTAL,HRSECONDARY_TOTAL,Labels,Primary_Total,Upper_Primary_Total,Secondary_Total,Higher_Secondary_Total,Labels,Drinking water,Electricity,Computer,Boys toilet,Girls toilet,Labels
Andaman & Nicobar Islands,0.800000,1.143333,7.543333,15.646667,3,100.216667,91.126667,78.386667,54.000000,0,98.69,88.86,53.06,94.52,93.44,0
Andhra Pradesh,4.750000,4.113333,13.693333,4.046667,0,91.260000,87.863333,93.243333,90.000000,6,90.35,90.34,29.57,56.88,81.31,0
Arunachal Pradesh,12.290000,6.590000,14.843333,13.533333,5,89.810000,81.456667,74.370000,57.000000,5,79.79,36.35,24.28,48.73,76.90,1
Assam,9.680000,8.253333,28.086667,3.976667,2,127.783333,122.200000,89.300000,64.666667,2,80.30,19.39,8.85,60.17,75.28,1
Bihar,0.696667,2.353333,27.123333,0.000000,1,111.500000,94.013333,74.526667,34.333333,1,92.53,9.96,5.47,73.42,75.41,1
Chandigarh,0.000000,0.540000,0.000000,11.826667,3,102.240000,97.733333,69.180000,29.666667,1,100.00,100.00,95.83,100.00,100.00,2
Chhattisgarh,2.823333,5.023333,19.843333,0.920000,4,86.800000,98.376667,89.663333,88.000000,6,95.47,57.28,9.80,82.85,94.10,1
Dadra & Nagar Haveli,1.013333,3.843333,23.256667,8.453333,5,102.363333,101.426667,97.246667,58.000000,0,100.00,96.79,38.19,85.53,94.36,0
Daman & Diu,0.653333,2.223333,21.216667,17.200000,5,83.496667,93.363333,86.766667,41.333333,1,100.00,100.00,61.11,100.00,100.00,0
Delhi,0.000000,1.180000,8.676667,16.013333,3,85.396667,83.280000,73.900000,34.000000,1,100.00,100.00,81.56,100.00,100.00,2
